## Installation
Using Python 3.12:
```
conda install jupyter numpy pip opencv matplotlib scipy
pip install pyvista[all]
pip install OpenVisus
python -m OpenVisus configure
```
Note: ignore errors during the "configure" process

## Required data
Download the following files to this directory:
```
HeadMRVolume.raw
T2.raw
ctscan_ez.vtk
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import OpenVisus as ov

from ipywidgets import fixed
import ipywidgets as widgets

from cannybase import get_edge_data

In [ ]:
def plot_volume(data, plotter=None, spacing=None):
    if plotter is None:
        plotter = pv.Plotter()
    else:
        plotter.clear()
    mesh = pv.wrap(data)
    if spacing:
        mesh.spacing = spacing
    plotter.add_volume(mesh)
    return plotter

In [ ]:
def interact_edge_data(data, thresholds, min_axes, gaussian, plotter, spacing=None):
    print(f'\rUpdating with {min_axes=}, {thresholds=}, {gaussian=}...', end='')
    edge_data = get_edge_data(data, thresholds[0], thresholds[1], min_axes, gaussian)
    plot_volume(edge_data, plotter, spacing)
    print(f'\rUpdated with {min_axes=}, {thresholds=}, {gaussian=}. Number of edge points: {edge_data.size - np.isnan(edge_data).sum()}.', end='')

In [ ]:
def make_widgets():
    threshold_widget = widgets.IntRangeSlider(
        min=0,
        max=255,
        value=(255,255),
        description='Canny Thresholds'
    )
    min_axes_widget = widgets.IntSlider(
        min=0,
        max=3,
        description='Min Axes'
    )
    gaussian_widget = widgets.FloatSlider(
        min=0,
        max=7,
        value=0,
        description='Gaussian SD'
    )
    return threshold_widget, min_axes_widget, gaussian_widget

## `HeadMRVolume.raw`

In [ ]:
headmr_data = np.fromfile('HeadMRVolume.raw', dtype='uint8')
headmr_data = headmr_data.reshape(42, 62, 48)

Original Data

In [ ]:
pl = plot_volume(headmr_data)
pl.show()

Edges Only

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget = make_widgets()
pl = plot_volume(headmr_data)
widgets.interact(
    interact_edge_data, 
    data=fixed(headmr_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    plotter=fixed(pl), 
    spacing=fixed(None)
)
pl.show()

## `T2.raw`

In [ ]:
t2_data = np.fromfile('T2.raw', dtype='float32')
t2_data = t2_data.reshape(320, 320, 256)

Original Data

In [ ]:
pl = plot_volume(t2_data)
pl.show()

Edges Only

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget = make_widgets()
pl = plot_volume(t2_data)
widgets.interact(
    interact_edge_data, 
    data=fixed(t2_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    plotter=fixed(pl), 
    spacing=fixed(None)
)
pl.show()

## `ctscan_ez.vtk`

In [ ]:
ctscan = pv.read('ctscan_ez.vtk')
ctscan

In [ ]:
ctscan.dimensions

Original Data

In [ ]:
pl = pv.Plotter()
pl.add_volume(ctscan)
pl.show()

In [ ]:
# Get the underlying data as a Numpy array
# PyVista stores the z-axis first, but we want the x-axis first in the Numpy array
ctscan_data = ctscan.point_data['image_data'].reshape(ctscan.dimensions[2], ctscan.dimensions[1], ctscan.dimensions[0])
ctscan_data = np.ndarray.copy(np.swapaxes(ctscan_data, 0, 2))

Edges Only

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget = make_widgets()
pl = plot_volume(ctscan_data, spacing=ctscan.spacing)
widgets.interact(
    interact_edge_data, 
    data=fixed(ctscan_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    plotter=fixed(pl), 
    spacing=fixed(ctscan.spacing)
)
pl.show()

## Kingsnake

In [ ]:
kingsnake_dataset = ov.load_dataset("https://klacansky.com/open-scivis-datasets/kingsnake/kingsnake.idx", cache_dir=".")
kingsnake_data = kingsnake_dataset.read(resolution=24)
spacing = (0.0688, 0.03174, 0.03174)

Original Data

In [ ]:
pl = plot_volume(kingsnake_data, spacing=spacing)
pl.show()

Edges Only

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget = make_widgets()
pl = plot_volume(kingsnake_data, spacing=spacing)
widgets.interact(
    interact_edge_data, 
    data=fixed(kingsnake_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    plotter=fixed(pl), 
    spacing=fixed(spacing)
)
pl.show()

## Engine

In [ ]:
engine_dataset = ov.load_dataset("https://klacansky.com/open-scivis-datasets/engine/engine.idx", cache_dir=".")
engine_data = engine_dataset.read(resolution=23)

Original Data

In [ ]:
pl = plot_volume(engine_data)
pl.show()

Edges Only

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget = make_widgets()
pl = plot_volume(engine_data, spacing=spacing)
widgets.interact(
    interact_edge_data, 
    data=fixed(engine_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    plotter=fixed(pl), 
    spacing=fixed(None)
)
pl.show()

## Bonsai

In [ ]:
bonsai_dataset = ov.load_dataset("https://klacansky.com/open-scivis-datasets/bonsai/bonsai.idx", cache_dir=".")
bonsai_data = bonsai_dataset.read(resolution=24)

Original Data

In [ ]:
pl = plot_volume(bonsai_data)
pl.show()

Edges Only

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget = make_widgets()
pl = plot_volume(bonsai_data)
widgets.interact(
    interact_edge_data, 
    data=fixed(bonsai_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    plotter=fixed(pl), 
    spacing=fixed(None)
)
pl.show()